# Исследование надежности заемщиков

Проведение исследования зависимости риска возникновения задолженности от различных факторов.

**Цель исследования.**

Нужно разобраться, влияет ли семейное положение, количество детей клиента и цели займа на факт погашения кредита в срок.

**Задачи исследования.**

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Общая информация

In [1]:
import pandas as pd

data = pd.read_csv(r'D:\Users\Egor\Desktop\Proga\Jupyter\portfolio\2_credit_scoring\credits.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставлю их как есть. Тем более этот столбец не понадобится для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

Создадим диапазоны, указанные ниже:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [22]:
def dependency_table(ind, val):
    data_pivot = data.pivot_table(index=ind, values=val, aggfunc=['sum', 'count', 'mean'])
    data_pivot = data_pivot.droplevel(1, axis=1).sort_values('mean')
    return data_pivot  

In [23]:
dependency_table('children', 'debt')

,sum,count,mean
children,,,
5,0,9,0.000000
0,1063,14091,0.075438
3,27,330,0.081818
1,444,4808,0.092346
2,194,2052,0.094542
4,4,41,0.097561


**Вывод:** заемщики без детей скорее с большей вероятностью выплатят кредит в срок, чем имеющие до 3 детей. Доля должников среди клиентов без детей составляет около 7,5%. У тех, кто имеет одноги или двух детей, доля невозврата 9,2 % и 9,4 соостветсвенно. Однако о клиентах, имеющих 4 и 5 детей, говорить тяжело, так как взята небольшая выборка.

In [24]:
dependency_table('family_status', 'debt')

,sum,count,mean
family_status,,,
вдовец / вдова,63,951,0.066246
в разводе,84,1189,0.070648
женат / замужем,927,12261,0.075606
гражданский брак,385,4134,0.093130
Не женат / не замужем,273,2796,0.097639


**Вывод:** трудно определить однозначную зависимость между семейным положением и возвратом кредита в срок. Можно сказать, что заемщики в разводе (7% доля невозрата), вдовцы/вдовы (6,6% доля невозрата) и женатые(7,6% доля невозрата) чаще возвращают кредит в срок, чем остальные группы. Ярче всего выделяются клиенты, находящиеся в гражданском браке или не женатые, у которых вероятность не выплатить кредит в срок около 9,5%.

In [25]:
dependency_table('total_income_category', 'debt')

,sum,count,mean
total_income_category,,,
D,21,349,0.060172
B,354,5014,0.070602
A,2,25,0.080000
C,1353,15921,0.084982
E,2,22,0.090909


**Вывод:** скорее всего, группа заемщиков, получающих от 200000 до 1000000, с большей вероятностью выплатит кредит в срок, чем группа с заработком 50000-200000. Группа B выплатит кредит не в срок с вероятностью 7%, в свою очередь группа С - 8,4%. Для групп клиентов с заработком до 50000 и выше 1000000 взята небольшая выборка. 

In [26]:
dependency_table('purpose_category', 'debt')

,sum,count,mean
purpose_category,,,
операции с недвижимостью,780,10751,0.072551
проведение свадьбы,183,2313,0.079118
получение образования,369,3988,0.092528
операции с автомобилем,400,4279,0.093480


**Вывод:** если заемщик берет кредит на опреции, связанные с недвижимостью или свадьбой, то с большей долей вероятности он вернет ссуду в срок, чем с опрециями, связанными с образовнием или автомобилем. Это может быть связано с тем, что на операции с недвижимостью берут часто большую сумму, которые нужно выплачивать долгий период времени, и заемщики стараются раньше погасить долг.

## Общий вывод

В ходе исследования сделаны основные шаги предобработки данных - заполнены пропуски, удалены аномальные значения, обработаны дубликаты, категоризированы данные нескольких столбцов. 

Сделаны выводы по основным вопросом данного исследования.

Чтобы расчеты и аналитика была точнее можно добавить данные, какую именно сумму денег заемщик не вернул в срок. Результаты исследования могут быть учтены при построении модели кредитного скоринга.